# END TO END ML USING SNOWPARK AND SCIKIT-LEARN

In this notebook we fit/train a Scikit-Learn ML pipeline that includes common feature engineering tasks such as Imputations, Scaling and One-Hot Encoding. The pipeline also includes a `RandomForestRegressor` model that will predict member lifetime value for ecommerce customers. 

We will fit/train the pipeline using a Snowpark Python Stored Procedure (SPROC) and then save the pipeline to a Snowflake stage. This example concludes by showing how a saved model/pipeline can be loaded and run in a scalable fashion on a snowflake warehouse using Snowpark Python User-Defined Functions (UDFs). 

![Snowpark ML](images/snowpark_ml.png)

### Create a session with Snowpark connector

In [5]:
# Snowpark
import snowflake.snowpark
import snowflake.snowpark.functions as F
from snowflake.snowpark.functions import sproc, udf, udtf, pandas_udf, col
from snowflake.snowpark.session import Session
from snowflake.snowpark import version as v
from snowflake.snowpark import types as T
from snowflake.snowpark.window import Window
import json

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


import pandas as pd
from cachetools import cached
import sys 
import numpy as np
import datetime
import io
import os
import joblib

In [2]:
with open('creds.json') as f:
    connection_parameters = json.load(f)

session = Session.builder.configs(connection_parameters).create()
session.add_packages('snowflake-snowpark-python', 'scikit-learn', 'pandas', 'numpy', 'joblib', 'cachetools')

The version of package cachetools in the local environment is 5.3.0, which does not fit the criteria for the requirement cachetools. Your UDF might not work when the package version is different between the server and your local environment


In [3]:
print(f"""
    ROLE: {session.get_current_role()}
    DATABASE: {session.get_current_database()}
    SCHEMA: {session.get_current_schema()}
    WAREHOUSE: {session.get_current_warehouse()}
""")


    ROLE: "SYSADMIN"
    DATABASE: "SNOWPARK_DEMO_DB"
    SCHEMA: "MEMBERSHIP_MODELING_DEMO"
    WAREHOUSE: "SNOWPARK_DEMO_WH"



### Create stages to save the ML model/pipeline and permanent UDFs

In [4]:
query = "create or replace stage models" +\
        " directory = (enable = true)" +\
        " copy_options = (on_error='skip_file')"

session.sql(query).collect()

[Row(status='Stage area MODELS successfully created.')]

In [ ]:
query = "create or replace stage udf" +\
        " copy_options = (on_error='skip_file')"

session.sql(query).collect()
session.file.put("preprocessing.zip", '@udf', auto_compress=False, overwrite=True)     

## Explore Snowflake data using Snowpark

In [7]:
snowdf = session.table("ECOMMERCE_CUSTOMERS_100K_JSON")
cat_attribs = ['GENDER','MEMBERSHIP_STATUS']
num_attribs = ['MEMBERSHIP_LENGTH', 'AVG_SESSION_LENGTH', 'TIME_ON_APP', 'TIME_ON_WEBSITE']
model_features = cat_attribs + num_attribs

In [8]:
snowdf.limit(100).to_pandas()

,JSON
0,"{\n ""AVG_SESSION_LENGTH"": 1.908812963760560e+..."
1,"{\n ""AVG_SESSION_LENGTH"": 2.051533536584287e+..."
2,"{\n ""AVG_SESSION_LENGTH"": 2.026216058545151e+..."
3,"{\n ""AVG_SESSION_LENGTH"": 2.052869179339092e+..."
4,"{\n ""AVG_SESSION_LENGTH"": 2.280240170727906e+..."
...,...
95,"{\n ""AVG_SESSION_LENGTH"": 2.727567630620153e+..."
96,"{\n ""AVG_SESSION_LENGTH"": 2.988700168582572e+..."
97,"{\n ""AVG_SESSION_LENGTH"": 8.309598674000878e+..."
98,"{\n ""AVG_SESSION_LENGTH"": 2.553961243389771e+..."


Add structure to our semi-structured data for analysis.

In [9]:
snowdf_normalized = snowdf.select(
     col("JSON")['GENDER'].cast(T.StringType()).as_('GENDER'),
     col("JSON")['MEMBERSHIP_STATUS'].cast(T.StringType()).as_('MEMBERSHIP_STATUS'),
     col("JSON")['MEMBERSHIP_LENGTH'].cast(T.FloatType()).as_('MEMBERSHIP_LENGTH'),
     col("JSON")['AVG_SESSION_LENGTH'].cast(T.FloatType()).as_('AVG_SESSION_LENGTH'),
     col("JSON")['TIME_ON_APP'].cast(T.FloatType()).as_('TIME_ON_APP'),
     col("JSON")['TIME_ON_WEBSITE'].cast(T.FloatType()).as_('TIME_ON_WEBSITE')
)

Or use some Python shortcuts

In [10]:
snowdf_normalized = snowdf.select(
        *[col("JSON")[f].astype(T.StringType()).as_(f) for f in cat_attribs],
        *[col("JSON")[f].astype(T.FloatType()).as_(f) for f in num_attribs],
)

In [11]:
snowdf_normalized.limit(100).to_pandas()

,GENDER,MEMBERSHIP_STATUS,MEMBERSHIP_LENGTH,AVG_SESSION_LENGTH,TIME_ON_APP,TIME_ON_WEBSITE
0,MALE,GOLD,1.876116,19.088130,197.696670,85.172298
1,MALE,GOLD,4.134936,20.515335,187.332228,107.133942
2,MALE,GOLD,2.172417,20.262161,218.383329,64.196458
3,UNKNOWN,GOLD,5.850095,20.528692,192.561488,70.309761
4,FEMALE,GOLD,9.494127,22.802402,222.214040,60.950384
...,...,...,...,...,...,...
95,MALE,GOLD,11.226694,27.275676,274.174034,50.271436
96,FEMALE,PLATIN,6.735049,29.887002,255.558311,114.085957
97,FEMALE,SILVER,2.513149,8.309599,87.123156,56.830521
98,FEMALE,GOLD,8.870128,25.539612,157.459003,35.213236


In [12]:
snowdf_normalized.describe().to_pandas()

,SUMMARY,GENDER,MEMBERSHIP_STATUS,MEMBERSHIP_LENGTH,AVG_SESSION_LENGTH,TIME_ON_APP,TIME_ON_WEBSITE
0,count,100000,100000,97651.000000,98571.000000,100000.000000,100000.000000
1,mean,None,None,4.516883,22.633535,180.404990,81.180553
2,stddev,None,None,2.274025,7.584742,70.979031,32.371648
3,max,UNKNOWN,SILVER,14.099797,54.616243,371.222991,174.284132
4,min,FEMALE,BASIC,0.000000,4.284894,-4.463542,-40.486104


In [13]:
snowdf_normalized.groupBy(col("MEMBERSHIP_STATUS")).avg(*num_attribs).to_pandas()

,MEMBERSHIP_STATUS,AVG(MEMBERSHIP_LENGTH),AVG(AVG_SESSION_LENGTH),AVG(TIME_ON_APP),AVG(TIME_ON_WEBSITE)
0,GOLD,4.468512,23.588624,185.928445,76.336760
1,BRONZE,0.930013,9.303863,74.747224,50.170735
2,PLATIN,6.082646,27.506596,245.330088,111.529437
3,DIAMOND,7.355709,32.544706,291.616519,119.468171
4,SILVER,2.983750,15.712558,98.666420,61.347554
5,BASIC,0.536478,6.589827,51.291637,49.636810


## Train ML Model
Create a stored procedure to push ML training to Snowpark

In [14]:
snowdf_train, snowdf_test = snowdf.random_split([0.8, 0.2], seed=82) # use seed to make the split repeatable

snowdf_train.write.mode("overwrite").save_as_table("MEMBERSHIP_TRAIN")
snowdf_test.write.mode("overwrite").save_as_table("MEMBERSHIP_TEST")

We use an sklearn pipeline for variable transformation. This entire pipeline gets serialized into `@MODELS/member_ltv_pipeline.joblib`, eliminating our need to do transformation on future datasets. 

In [15]:
%%writefile transformers/json_normalizer.py
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class JSON_Normalizer(BaseEstimator, TransformerMixin):
    #the constructor
    def __init__(self):
        None
    #estimator method
    def fit(self, X, y = None):
        return self
    #transfprmation
    def transform(self, X, y = None):
        #getting the three extra attributes by dividing appropriate attributes
        return pd.json_normalize(X)

Overwriting transformers/json_normalizer.py


In [16]:
def train_model(session: snowflake.snowpark.Session):
        
    from transformers.json_normalizer import JSON_Normalizer


    ## take JSON and create structured table for training
    snowdf = session.table("ECOMMERCE_CUSTOMERS_10K_JSON")

    # split the train and test set
    snowdf_train, snowdf_test = snowdf.random_split([0.8, 0.2], seed=82) # use seed to make the split repeatable

    # save the train and test sets as time stamped tables in Snowflake 
    snowdf_train.write.mode("overwrite").save_as_table("MEMBERSHIP_TRAIN")
    snowdf_test.write.mode("overwrite").save_as_table("MEMBERSHIP_TEST")

    # X datasets load raw JSON records which will be pivoted to columns as part of the pipeline
    # this allows your pipeline to take raw JSON with no pre-transforms
    X_train = snowdf_train.to_pandas()['JSON'].apply(json.loads)
    Y_train = pd.json_normalize(snowdf_train.to_pandas()['JSON'].apply(json.loads))['YEARLY_SPENT']
    X_test = snowdf_test.to_pandas()['JSON'].apply(json.loads)
    Y_test = pd.json_normalize(snowdf_test.to_pandas()['JSON'].apply(json.loads))['YEARLY_SPENT']

    
    json_normalizer = JSON_Normalizer()
    
    # create a pipeline for numerical features
    num_pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy="median")),
            ('std_scaler', StandardScaler())
        ])

    cat_pipeline = Pipeline([
        ("cat", OneHotEncoder())
    ])

    preprocessor = ColumnTransformer([
            ("num", num_pipeline, num_attribs),
            ("cat", cat_pipeline, cat_attribs)
        ])

    full_pipeline = Pipeline([
            ('json_norm', json_normalizer),
            ('preprocessor', preprocessor),
            ('model', RandomForestRegressor(n_estimators=100, random_state=42)),
        ])

    # fit the preprocessing pipeline and the model together
    full_pipeline.fit(X_train, Y_train)

    # save the full pipeline including the model
    from joblib import dump
    dump(full_pipeline, '/tmp/member_ltv_pipeline.joblib')
    session.file.put('/tmp/member_ltv_pipeline.joblib', '@MODELS', auto_compress=False, overwrite=True)

    # predict on the test set and return the root mean squared error (RMSE)
    Y_pred = full_pipeline.predict(X_test)
    lin_mse = mean_squared_error(Y_test, Y_pred)
    lin_rmse = np.sqrt(lin_mse)
    return lin_rmse

# Create an instance of StoredProcedure using the sproc() function
train_model_sp = sproc(train_model, replace=True, imports=['transformers'], return_type=T.FloatType())

### Use high-memory warehouse for fitting our model.

In [17]:
#use a snowpark-optimized high memory warehouse for model fitting
session.sql("USE WAREHOUSE snowpark_demo_wh_high_mem").collect()

# run model training stored procedure
rmse = train_model_sp()

# switch back to standard warehouse
session.sql("USE WAREHOUSE snowpark_demo_wh").collect()

print(f"RMSE: {rmse}")

RMSE: 47.0296565174487


In [18]:
session.sql('list  @models').collect()

[Row(name='models/member_ltv_pipeline.joblib', size=65481920, md5='6462a86041c24be51fc5408abb027e10', last_modified='Thu, 2 Feb 2023 15:49:56 GMT')]

### Retrieve our model for additional analysis.

In [19]:
def retrieve_model(stage_location):
    import gzip 
    filename = stage_location.split('/')[-1]
    session.file.get(stage_location, '.')
    model_file = joblib.load(filename)

    return model_file

pipeline = retrieve_model("@MODELS/member_ltv_pipeline.joblib")
pipeline

Pipeline(steps=[('json_norm', JSON_Normalizer()),
                ('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('std_scaler',
                                                                   StandardScaler())]),
                                                  ['MEMBERSHIP_LENGTH',
                                                   'AVG_SESSION_LENGTH',
                                                   'TIME_ON_APP',
                                                   'TIME_ON_WEBSITE']),
                                                 ('cat',
                                                  Pipeline(steps=[('cat',
                                                                   OneHotEncoder())]),
                                                  ['GENDER',
                                                   'MEMBERSHIP_STATUS'])])),
                ('model', RandomForestRegressor(random_state=42))])

In [20]:
rfr = pipeline.named_steps['model']

cat_attribs_transformed = list(pipeline.named_steps['preprocessor'].transformers_[1][1]\
   .named_steps['cat'].get_feature_names_out(cat_attribs))

transformed_feature_names = num_attribs + cat_attribs_transformed 

pd.DataFrame(rfr.feature_importances_,
             index=transformed_feature_names, 
             columns=["FEAT_IMPORTANCE"]).sort_values("FEAT_IMPORTANCE", ascending=False)

,FEAT_IMPORTANCE
TIME_ON_APP,0.763375
MEMBERSHIP_STATUS_DIAMOND,0.042592
AVG_SESSION_LENGTH,0.031576
TIME_ON_WEBSITE,0.030301
MEMBERSHIP_STATUS_BASIC,0.029173
MEMBERSHIP_LENGTH,0.026456
MEMBERSHIP_STATUS_BRONZE,0.023593
MEMBERSHIP_STATUS_PLATIN,0.023002
MEMBERSHIP_STATUS_GOLD,0.021444
MEMBERSHIP_STATUS_SILVER,0.004554


In [21]:
sample_json = {"AVG_SESSION_LENGTH": 1.908812963760560e+01,
               "EMAIL": "4jx6famntt@v8caw.com",
               "GENDER": "MALE",
               "MEMBERSHIP_LENGTH": 1.876116082413728e+00,
               "MEMBERSHIP_STATUS": "SILVER",
               "TIME_ON_APP": 1.976966698388652e+02,
               "TIME_ON_WEBSITE": 2.517229813055825e+01}
pipeline.predict(sample_json)

array([386.14759703])

## Deploy Model as UDF

#### Option 1: Deploy as UDF

In [22]:
@cached(cache={})
def read_file(filename):
    import_dir = sys._xoptions.get("snowflake_import_directory")
    m = joblib.load(os.path.join(import_dir, filename))
    
    return m

def predict(JSON_VALUE):
       m = read_file("member_ltv_pipeline.joblib")
       return m.predict(JSON_VALUE)[0]

pred_udf = udf(predict, 
               name="predict_pipeline", 
               is_permanent=True, 
               input_types=[T.VariantType()],
               return_type = T.FloatType(),
               stage_location="@udf", 
               replace=True, 
               imports=['transformers', '@MODELS/member_ltv_pipeline.joblib'])

In [23]:
inference_df = session.table("ECOMMERCE_CUSTOMERS_100K_JSON")
scored_df = inference_df.select(col("JSON")["EMAIL"].alias("EMAIL"),
                                col("JSON"), 
                                F.call_udf("predict_pipeline", 
                                           col("JSON")
                                          ).alias("PREDICTED_YEARLY_SPENT")
                               ).limit(100).to_pandas()

In [24]:
scored_df

,EMAIL,JSON,PREDICTED_YEARLY_SPENT
0,"""4jx6famntt@v8caw.com""","{\n ""AVG_SESSION_LENGTH"": 1.908812963760560e+...",422.839122
1,"""fapk6sr7rk@bfjyn.com""","{\n ""AVG_SESSION_LENGTH"": 2.051533536584287e+...",466.839576
2,"""gkmyoxydtg@lfsco.com""","{\n ""AVG_SESSION_LENGTH"": 2.026216058545151e+...",450.849914
3,"""nrcdihhzp0@920gu.com""","{\n ""AVG_SESSION_LENGTH"": 2.052869179339092e+...",458.531145
4,"""zikblq3pfe@fm6t9.com""","{\n ""AVG_SESSION_LENGTH"": 2.280240170727906e+...",509.439224
...,...,...,...
95,"""frgw4qvqr7@inpph.com""","{\n ""AVG_SESSION_LENGTH"": 2.727567630620153e+...",530.810840
96,"""0gedokamrd@0csjm.com""","{\n ""AVG_SESSION_LENGTH"": 2.988700168582572e+...",595.755925
97,"""hc0lmy3jr2@okghl.com""","{\n ""AVG_SESSION_LENGTH"": 8.309598674000878e+...",280.476326
98,"""e51a9zvp9u@z9gmp.com""","{\n ""AVG_SESSION_LENGTH"": 2.553961243389771e+...",510.890309


#### Option 2: Deploy as Vectorized UDF
We're using a vectorized UDF, which automatically splits up the rows and sends a batch to each UDF execution resulting in better throughput. Additionally, we're caching the model load from stage to decrease IO cost. 

In [ ]:
session.add_import("@MODELS/member_ltv_pipeline.joblib")  

def read_file(filename):
    import_dir = sys._xoptions.get("snowflake_import_directory")
    m = joblib.load(os.path.join(import_dir, filename))
    
    return m

@pandas_udf(name="predict_pipeline_batch", 
            stage_location="@udf", 
            max_batch_size=1000, 
            is_permanent=True, 
            replace=True,
            imports=['transformers', '@MODELS/member_ltv_pipeline.joblib'])
def predict_pipeline_batch(df: T.PandasSeries[dict]) -> T.PandasSeries[float]:
       m = read_file("member_ltv_pipeline.joblib") 
       return m.predict(df)

### Run Inference UDF on 1 million records

Now running the vectorized UDF

In [ ]:
inference_df = session.table("ECOMMERCE_CUSTOMERS_1M_JSON")

scored_df = inference_df.select("*", 
                                F.call_udf("predict_pipeline_batch", 
                                            col("JSON")
                                          ).alias("PREDICTED_YEARLY_SPENT")
                               )
scored_df.write.mode("OVERWRITE").save_as_table("INFERED_DATA_1M")
    
session.table("INFERED_DATA_1M").limit(100).to_pandas()

You could also run this in SQL

In [ ]:
session.sql(""" create temp table pred_1m as 
                select predict_pipeline_batch(a.JSON) as prediction, 
                       a.* 
                from ECOMMERCE_CUSTOMERS_1M_JSON a""").collect()

In [ ]:

session.sql("""select * from pred_1m limit 100""").to_pandas()